In [243]:
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import requests
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
from pandas import ExcelWriter
import glob
import os

In [244]:
#loop through all the eviction-date data file names in a given folder
os.chdir(r"C:\Users\wjg39\Documents\Desktop\AP Evictions")
FileList = glob.glob('*.xlsx')
print(FileList)

['week-ending-2020-10-24.xlsx', 'week-ending-2020-10-31.xlsx', 'week-ending-2020-11-07.xlsx', 'week-ending-2020-11-14.xlsx', 'week-ending-2020-11-21.xlsx', 'week-ending-2020-11-28.xlsx', 'week-ending-2020-12-05.xlsx', 'week-ending-2020-12-12.xlsx', 'week-ending-2020-12-19.xlsx', 'week-ending-2020-12-26.xlsx', 'week-ending-2021-01-02.xlsx', 'week-ending-2021-01-09.xlsx', 'week-ending-2021-01-16.xlsx', 'week-ending-2021-01-23.xlsx', 'week-ending-2021-01-30.xlsx', 'week-ending-2021-02-06.xlsx', 'week-ending-2021-02-13.xlsx', 'week-ending-2021-02-20.xlsx', 'week-ending-2021-02-27.xlsx']


In [245]:
#dating the end of the date from the file name to rename columns
column_name=FileList
column_name=[new_name[12:22] for new_name in column_name]

In [246]:
df = pd.DataFrame(columns=['Code']) 

#Start first dataframe then append times series data to it
Municipality_tables=pd.read_excel(FileList[0], "Municipality rate per 100k")

#loop to merge data through an outer join
for index in range(1,len(FileList)):
    dates=pd.read_excel(FileList[index], "Municipality rate per 100k")
    Municipality_tables=Municipality_tables.merge(dates, on='Municipality', how='outer',suffixes=(' '+column_name[index-1],' '+column_name[index]))

#because suffixes only works if there is an even number of dates, manually rename if odd number
if(len(FileList)%2==1):
    Municipality_tables.rename(columns={  Municipality_tables.columns[-2]: "Residents "+column_name[-1] }, inplace = True)
    Municipality_tables.rename(columns={  Municipality_tables.columns[-1]: "Count "+column_name[-1] }, inplace = True)

In [247]:
#sort table in alphabetical order by municipality
Municipality_tables=Municipality_tables.sort_values('Municipality')
Municipality_tables

,Municipality,Residents 2020-10-24,Count 2020-10-24,Residents 2020-10-31,Count 2020-10-31,Residents 2020-11-07,Count 2020-11-07,Residents 2020-11-14,Count 2020-11-14,Residents 2020-11-21,...,Residents 2021-01-30,Count 2021-01-30,Residents 2021-02-06,Count 2021-02-06,Residents 2021-02-13,Count 2021-02-13,Residents 2021-02-20,Count 2021-02-20,Residents 2021-02-27,Count 2021-02-27
32,(not given),0.0,2.0,0.0,5.0,0.0,1.0,0.0,23.0,0.0,...,0.0,243.0,0.0,269.0,0.0,233.0,0.0,211.0,0.0,167.0
185,Abington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,2.0,6.0,1.0
235,Acton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,Acushnet,NaN,NaN,NaN,NaN,NaN,NaN,9.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,Adams,NaN,NaN,NaN,NaN,NaN,NaN,23.0,2.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Worcester,1.0,2.0,3.0,7.0,4.0,8.0,24.0,44.0,28.0,...,12.0,22.0,9.0,17.0,22.0,40.0,4.0,9.0,8.0,16.0
324,Wrentham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,27.0,3.0,NaN,NaN,NaN,NaN,9.0,1.0
37,Yarmouth Port,NaN,NaN,19.0,1.0,19.0,1.0,NaN,NaN,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [248]:
#gather ERMA data
ERMA_Link='https://docs.google.com/spreadsheets/d/1370AuhG6Ktms2tshEmFNN1cOqXQt-g5bcpAnPox7ZcA/export?format=csv&gid=0'
ERMA_database=pd.read_csv(ERMA_Link,skiprows=2)

#data cleaning any extra spaces so can merge by Municiplaity
ERMA_database.columns=ERMA_database.columns.str.strip()

In [249]:
#Merge ERMA data with masslandlords data
merged_DB=ERMA_database.merge(Municipality_tables,on='Municipality',how='left')
merged_DB

,Municipality,Administering Agency,Amount Allocated,Funding Source,Max $ Assistance per household (total),"Household Assistance Details (# of months, max per month, etc)",Area Median Income Elgibility,Program Timeline,Contact info,Link to application,...,Residents 2021-01-30,Count 2021-01-30,Residents 2021-02-06,Count 2021-02-06,Residents 2021-02-13,Count 2021-02-13,Residents 2021-02-20,Count 2021-02-20,Residents 2021-02-27,Count 2021-02-27
0,Acton,Regional Housing Services Office (RHSO),"$55,000",CARES Act,"$3,200","Up to 4 months of assistance, $ amount based o...",Households up to 100% AMI,Not specified,"Laura Ducharme, lducharme@actonma.gov",https://www.rhsohousing.org/sites/g/files/vyhl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amesbury,The Pettengill House,Not specified,Not specified,Not specified,Not specified,Not specified,Not specified,Not specified,http://pettengillhouse.org/aid4amesbury.aspx,...,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amherst,Community Action Pioneer Valley,"$250,000",Trust,"$3,300","Up to 3 months of assistance, $ amount based o...",Households up to 80% AMI,Until funds are depleted.,"Janna Tetreault, Assistant Director for Commun...",https://www.communityaction.us/amherstrentalas...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arlington,"Town or Arlington, Housing Corp or Arlington, ...","$400,000 CDBG-CV funds, $300,000 CPA (starting...","CDBG-CV, CPA, regular CDBG, and donations","$6,000",Varies on funding source. CBDG-CV is up to thr...,Households up to 50% AMI. Housing Corp of Arli...,Through the fall.,"Erin Zwirko, Arlington Planning and Community ...",https://www.arlingtonma.gov/departments/health...,...,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,4.0,2.0
4,Barnstable County,Housing Assistance Cape Cod (HAC),"$100,000",HOME,"$6,000",Not specified,Households up to 60% AMI,Launched fall 2020,"Renie Hamman, Barnstable County HOME Consortium",https://www.cognitoforms.com/HousingAssistance...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Weymouth,NeighborWorks Housing Solutions,"239,773",CDBG-CV,"$4,000",Up to 3 months,Households up to 80% AMI,12 months from start of contract or until fund...,"Christine Howe, Grants and Procurement, chowe@...",Under Development,...,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,1.0,1.0
74,Williamstown,Berkshie Housing Development Corp.,"$20,700 Trust, $75,000 CPA",Trust/CPA,"$500 - or $1,000 in extreme situations",Not specified,Households up to 100% AMI,Not specified,"Tom Sheldon, tom.e.sheldon@gmail.com",https://williamstownma.gov/2020/05/14/rental-a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,Yarmouth,Housing Assistance Cape Cod (HAC) and Hands Of...,"$108,000","Trust, CPA","$5,000",Not specified,Households up to 100% AMI,Various programs expiring between May and June...,"Mary Waygan, Affordable Housing/CDBG Program A...",HAC Funds; https://haconcapecod.org/blog/workf...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,"South Berkshire Consortium, see notes*",Construct Berkshire,"$220,000",Foundation funding,"$1,000",Not specified,Households up to 80% AMI,Rolling,"June Wolfe, Housing Director, jwolfe@construct...",https://constructinc.org/programs/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
#export dataframe to csv
merged_DB.to_csv(r'C:\Users\wjg39\Documents\Desktop\AP Evictions\merged_eviction_data.csv',index=False)
Municipality_tables.to_csv(r'C:\Users\wjg39\Documents\Desktop\AP Evictions\masslandlords.csv',index=False)